<a href="https://colab.research.google.com/github/laduqueo/Proyecto_Final_SyS/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#instalación de librerías
!pip install streamlit -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
!mkdir pages

In [ ]:
%%writefile 0_Introducción.py

import streamlit as st

st.title("De Fourier al WiFi/5G")

st.markdown("""
**¡Bienvenid@ al proyecto final de Señales y Sistemas 2025!**
Aquí vas a descubrir, paso a paso, cómo las ideas básicas del análisis de señales nos ayudan a construir tecnologías que usamos todos los días, como **WiFi y 5G**.

Este dashboard interactivo te permite:

- Ver cómo se comportan las señales en el **tiempo y la frecuencia**.
- Aplicar **filtros digitales** para eliminar ruido.
- Explorar señales **I/Q** con la **Transformada de Hilbert**.
- Simular la **modulación QAM** y ver su **constelación** en acción.
- Probar un canal con **ruido (AWGN)** y ver cómo afecta la señal recibida.
---

### Desarrollado por:

- **Esteban Becerra Loaiza**
- **Laura Lorena Duque Ospina**
- **José Luis Ortiz Álvarez**

""")


Writing 0_Introducción.py


In [ ]:
%%writefile 1_Conceptos_clave.py

import streamlit as st

st.title("Conceptos Clave")

# Tabs para cada tema
tabs = st.tabs([
    "Transformada de Fourier",
    "Filtros FIR/IIR",
    "Transformada de Hilbert",
    "Señales I/Q y QAM",
    "OFDM",
    "WiFi y 5G"
])

with tabs[0]:
    st.markdown("""
    # 1. Transformada de Fourier

    ## ¿Qué hace la Transformada de Fourier?

    Descompone una señal en sus componentes de frecuencia. Permite analizar qué frecuencias están presentes en una señal y con qué intensidad.

    Por ejemplo, una señal de audio compleja puede estar compuesta por múltiples tonos (frecuencias) que se suman. La Transformada de Fourier nos permite ver esos tonos.

    ## Definición matemática (caso continuo):

    $$X(f) = \int_{-\infty}^{\infty} x(t) \cdot e^{-j2\pi ft} \, dt$$

    - \(x(t)\) es la señal en el tiempo.
    - \(X(f)\) es su representación en el dominio de la frecuencia.
    - \(f\) es la frecuencia (Hz).

    ## ¿Y en el caso digital?

    Usamos la **Transformada Discreta de Fourier (DFT)** o su versión rápida **FFT**.

    $$X[k] = \sum_{n=0}^{N-1} x[n] \cdot e^{-j2\pi kn/N}$$

    Donde:

    - \(x[n]\) es la señal discreta.
    - \(X[k]\) es la amplitud en la \(k\)-ésima frecuencia.

    ## ¿Para qué sirve en comunicaciones?

    - Analizar espectro de señales moduladas.
    - Diseñar filtros.
    - Implementar modulación OFDM.
    - Detectar componentes no deseadas (ruido, interferencias).

    ## Ejemplo práctico
    """)

    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.fft import fft, fftfreq

    fs = 1000
    T = 1.0
    N = int(fs * T)
    t = np.linspace(0.0, T, N, endpoint=False)
    x = np.sin(2*np.pi*50*t) + 0.5*np.sin(2*np.pi*120*t)

    xf = fftfreq(N, 1/fs)[:N//2]
    yf = fft(x)
    magnitude = 2.0/N * np.abs(yf[0:N//2])

    fig, axs = plt.subplots(2, 1, figsize=(10, 6))

    axs[0].plot(t, x, color='darkblue')
    axs[0].set_title("Señal en el tiempo")
    axs[0].set_xlabel("Tiempo [s]")
    axs[0].set_ylabel("Amplitud")
    axs[0].grid(True)

    axs[1].stem(xf, magnitude, linefmt='darkorange', markerfmt='ro', basefmt=' ')
    axs[1].set_title("Espectro de Fourier")
    axs[1].set_xlabel("Frecuencia [Hz]")
    axs[1].set_ylabel("Magnitud")
    axs[1].grid(True)

    fig.tight_layout()
    st.pyplot(fig)

    st.markdown("""
    ## Observación

    Vemos dos picos claros en el espectro: uno a 50 Hz y otro a 120 Hz. Eso confirma que la señal contiene esas dos frecuencias.

    Esta técnica es la base para todo análisis de espectro en comunicaciones digitales.
    """)

with tabs[1]:
    st.markdown("""
   # 2. Filtrado Digital (FIR / IIR)

    Un filtro digital modifica el contenido en frecuencia de una señal. Se utiliza para eliminar ruido, destacar frecuencias específicas o suavizar señales. Se clasifican en FIR (Respuesta Finita al Impulso) e IIR (Respuesta Infinita al Impulso).

    ## FIR
    - No usa salidas anteriores.
    - Siempre estable.
    - Se puede diseñar con fase lineal.
    - Ecuación:
      $$y[n] = \sum_{k=0}^{N} b_k \cdot x[n - k]$$

    ## IIR
    - Usa entradas y salidas anteriores.
    - Más eficiente, pero puede ser inestable.
    - Ecuación:
      $$y[n] = \sum_{k=0}^{M} b_k \cdot x[n - k] - \sum_{k=1}^{N} a_k \cdot y[n - k]$$

    ## Tipos de filtros
    - Pasa-bajo: atenúa altas frecuencias.
    - Pasa-alto: atenúa bajas frecuencias.
    - Pasa-banda: deja pasar un rango de frecuencias.
    - Rechaza-banda: elimina un rango específico.

    ## Ejemplo de filtrado FIR (suavizado)
    """)

    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import firwin, lfilter

    fs = 1000
    t = np.linspace(0, 1.0, fs, endpoint=False)
    x = np.sin(2*np.pi*50*t) + 0.5*np.sin(2*np.pi*250*t)

    cutoff = 100
    numtaps = 101
    fir_coeff = firwin(numtaps=numtaps, cutoff=cutoff, fs=fs, window='hamming')
    y = lfilter(fir_coeff, 1.0, x)

    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, x, label='Original', alpha=0.6)
    ax.plot(t, y, label='Filtrada (FIR)', linewidth=2)
    ax.set_title("Señal original vs. filtrada")
    ax.set_xlabel("Tiempo [s]")
    ax.set_ylabel("Amplitud")
    ax.grid(True)
    ax.legend()
    st.pyplot(fig)

with tabs[2]:

    st.markdown(" # 3. Señales Analíticas y Transformada de Hilbert")
    st.markdown(r"""

    La **Transformada de Hilbert** es una herramienta que permite construir una versión compleja de una señal real. Esta nueva señal, llamada **señal analítica**, es útil para analizar fase y amplitud de forma más precisa en procesamiento digital de señales.

    ## ¿Qué hace?

    Dada una señal real $ x(t) $, la Transformada de Hilbert produce otra señal $\hat{x}(t)$ que está desfasada $90^\circ$ respecto a la original.

    Así se puede formar la señal analítica:

    $$
    z(t) = x(t) + j\hat{x}(t)
    $$

    Esta combinación permite calcular:

    - **Envolvente**:
      $$
      |z(t)| = \sqrt{x^2(t) + \hat{x}^2(t)}
      $$
    - **Fase instantánea**:
      $$
      \phi(t) = \arg(z(t)) = \arctan\left(\frac{\hat{x}(t)}{x(t)}\right)
      $$

    ## ¿Para qué sirve?

    Es fundamental en:
    - Modulación de amplitud (AM-SSB)
    - Análisis I/Q (in-phase / quadrature)
    - Medición de fase y frecuencia instantánea
    - Procesamiento de señales biomédicas y radar

    A continuación se ilustra cómo se obtiene la señal analítica y sus propiedades.
    """)

    import numpy as np
    import matplotlib.pyplot as plt
    from scipy.signal import hilbert

    fs = 1000
    t = np.linspace(0, 1, fs, endpoint=False)
    f = 50
    x = np.sin(2 * np.pi * f * t)

    z = hilbert(x)
    xh = np.imag(z)
    envolvente = np.abs(z)
    fase = np.angle(z)

    azul_oscuro = '#002744'
    naranja_oscuro = '#cc5500'
    rojo_oscuro = '#8b0000'
    verde_oscuro = '#006400'

    fig, axs = plt.subplots(3, 1, figsize=(12, 8))

    axs[0].plot(t, x, label='x(t) - señal real', color=azul_oscuro)
    axs[0].plot(t, xh, label='ẋ(t) - Hilbert', linestyle='--', color=naranja_oscuro)
    axs[0].set_title('Transformada de Hilbert: señal real y en cuadratura')
    axs[0].set_xlabel('Tiempo [s]')
    axs[0].grid(True)
    axs[0].legend()

    axs[1].plot(t, x, label='x(t)', color=azul_oscuro)
    axs[1].plot(t, envolvente, label='Envolvente |z(t)|', color=rojo_oscuro)
    axs[1].set_title('Envolvente de la señal analítica')
    axs[1].set_xlabel('Tiempo [s]')
    axs[1].grid(True)
    axs[1].legend()

    axs[2].plot(t, fase, label='Fase instantánea φ(t)', color=verde_oscuro)
    axs[2].set_title('Fase instantánea de la señal')
    axs[2].set_xlabel('Tiempo [s]')
    axs[2].grid(True)
    axs[2].legend()

    fig.tight_layout()
    st.pyplot(fig)

    st.markdown(r"""
    ## Observaciones finales

    - La señal en cuadratura $$\hat{x}(t)$$ es como una versión girada de la original.
    - La envolvente permite observar la amplitud global de una señal modulada.
    - La fase instantánea revela la evolución de frecuencia y desplazamientos sutiles.

    Estas tres propiedades son muy útiles para estudiar señales reales con variaciones rápidas o complejas.
    """)

with tabs[3]:
    st.markdown(""" # 4. Señales I/Q y Modulación QAM

    ## ¿Qué son las señales I/Q?

    Las señales I/Q (In-Phase y Quadrature) permiten representar una señal como la combinación de dos componentes ortogonales: una cosenoidal (I) y otra senoidal desfasada 90° (Q). Esto permite modular tanto amplitud como fase.

    ## ¿Qué es QAM?

    QAM (Quadrature Amplitude Modulation) combina variaciones en amplitud y fase para codificar datos. Cada combinación única de I y Q representa un símbolo distinto, mostrado como un punto en una constelación.

    ## Bits por símbolo

    | Tipo de QAM   | Bits por símbolo | Puntos en la constelación |
    |---------------|------------------|----------------------------|
    | 4-QAM         | 2 bits           | 4                          |
    | 16-QAM        | 4 bits           | 16                         |
    | 64-QAM        | 6 bits           | 64                         |
    | 256-QAM       | 8 bits           | 256                        |

    A mayor número de puntos, mayor eficiencia espectral, pero también mayor sensibilidad al ruido.
    """, unsafe_allow_html=True)

    import numpy as np
    import matplotlib.pyplot as plt

    niveles = [-3, -1, 1, 3]
    N = 1000
    I = np.random.choice(niveles, size=N)
    Q = np.random.choice(niveles, size=N)
    s = I + 1j * Q

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot(np.real(s), np.imag(s), 'o', color='#003f5c', alpha=0.4, label='Símbolos QAM')
    ax.axhline(0, color='gray', linewidth=1)
    ax.axvline(0, color='gray', linewidth=1)

    for level in niveles:
        ax.axhline(level, color='lightgray', linestyle='--', linewidth=0.5)
        ax.axvline(level, color='lightgray', linestyle='--', linewidth=0.5)

    ax.text(np.real(s[0]) + 0.2, np.imag(s[0]) + 0.2, f"({I[0]}, {Q[0]})", fontsize=9, color='black')
    ax.set_title(f'Constelación 16-QAM simulada ({len(set(s))} puntos únicos)')
    ax.set_xlabel('I (En fase)')
    ax.set_ylabel('Q (En cuadratura)')
    ax.grid(True)
    ax.axis('equal')
    ax.legend()
    st.pyplot(fig)

    st.markdown("""
    ## ¿Qué representa la gráfica?

    Cada punto azul es un símbolo distinto definido por su componente I (horizontal) y Q (vertical).

    A mayor número de niveles en I y Q, mayor será la cantidad de símbolos distintos (más bits por símbolo), pero también más sensibles al ruido.

    Esta técnica se usa ampliamente en tecnologías como 5G, WiFi y televisión digital.
    """
    )

# --- Tab 5: OFDM ---
with tabs[4]:
    st.markdown(r"""
    # 5. OFDM – Multiplexación por División de Frecuencia Ortogonal

    OFDM (Orthogonal Frequency Division Multiplexing) es una técnica de modulación digital avanzada usada en WiFi, LTE, 5G y otros sistemas modernos. Divide el canal disponible en múltiples subportadoras ortogonales para transmitir datos en paralelo.

    En lugar de usar una sola frecuencia como en AM o FM, OFDM reparte la información en muchas frecuencias pequeñas. Esto mejora la resistencia al ruido y la eficiencia del espectro.

    ## ¿Por qué se llama ortogonal?

    Las subportadoras están espaciadas cuidadosamente para que sean ortogonales, es decir, que no interfieran entre sí aunque se solapen. Matemáticamente:

    $$
    \int_0^T \cos(2\pi f_i t) \cdot \cos(2\pi f_j t)\,dt = 0 \quad \text{si } i \ne j
    $$

    Esto maximiza la eficiencia sin interferencia.

    ## Ventajas clave:

    - Alta eficiencia espectral.
    - Resistente a interferencias y desvanecimientos.
    - Ideal para entornos urbanos con rebotes.
    - Compatible con técnicas como QAM y FFT para transmisión y recepción.
    """)

    import numpy as np
    import matplotlib.pyplot as plt

    N = 64
    niveles = [-3, -1, 1, 3]
    I = np.random.choice(niveles, size=N)
    Q = np.random.choice(niveles, size=N)
    X = I + 1j * Q
    idx = np.random.randint(N)
    symbol_original = X[idx]

    ofdm_signal = np.fft.ifft(X)
    t = np.linspace(0, 1, N, endpoint=False)

    plt.figure(figsize=(10, 6))
    plt.plot(t, np.real(ofdm_signal), label='Parte real', color='navy')
    plt.plot(t, np.imag(ofdm_signal), label='Parte imaginaria', color='darkorange', linestyle='--')
    plt.title('Señal OFDM (Parte Real e Imaginaria)')
    plt.xlabel('Tiempo')
    plt.ylabel('Amplitud')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    st.pyplot(plt.gcf())

    received_signal = ofdm_signal
    X_rec = np.fft.fft(received_signal)

    plt.figure(figsize=(10, 6))
    plt.stem(np.abs(X), linefmt='g-', markerfmt='go', basefmt=' ', label='Espectro original')
    plt.stem(np.abs(X_rec), linefmt='r--', markerfmt='ro', basefmt=' ', label='Espectro recuperado')
    plt.title('Espectro: Original vs Recuperado (FFT)')
    plt.xlabel('Subportadora')
    plt.ylabel('Magnitud')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    st.pyplot(plt.gcf())

    symbol_received = X_rec[idx]

    plt.figure(figsize=(10, 6))
    plt.plot(np.real(symbol_original), np.imag(symbol_original), 'bo', label='Original')
    plt.plot(np.real(symbol_received), np.imag(symbol_received), 'rx', label='Recuperado')
    plt.title('Símbolo QAM: Original vs Recuperado')
    plt.xlabel('Parte Real')
    plt.ylabel('Parte Imaginaria')
    plt.grid(True)
    plt.axis('equal')
    plt.legend()
    plt.tight_layout()
    st.pyplot(plt.gcf())


with tabs[5]:
    st.markdown("""
    # 6. Comunicaciones WiFi y 5G

    ## ¿Qué son WiFi y 5G?

    WiFi y 5G son tecnologías que permiten la transmisión de datos de forma inalámbrica usando ondas electromagnéticas. Aunque se usan en contextos distintos (WiFi en redes locales como hogares u oficinas, y 5G en redes móviles de amplio alcance), ambas se basan en principios técnicos similares que permiten transmitir gran cantidad de información de forma eficiente, rápida y confiable.

    Ambas tecnologías utilizan:
    - Modulación digital por amplitud en cuadratura (QAM)
    - Multiplexación mediante división ortogonal de frecuencias (OFDM)
    - Representación en componentes I (en fase) y Q (en cuadratura)
    - Transformadas rápidas de Fourier (FFT e IFFT)
    - Filtrado digital para reducir interferencias

    Estas técnicas permiten que múltiples usuarios transmitan datos simultáneamente, con alta velocidad y baja latencia.

    ## ¿Cómo funciona el proceso de transmisión?

    1. **Codificación y Modulación QAM:**
    Los bits se agrupan en bloques y se convierten en símbolos QAM.

    2. **Multiplexación OFDM:**
    Los símbolos se asignan a subportadoras ortogonales. Se aplica la IFFT para pasar al dominio temporal.

    3. **Generación de señales I/Q:**
    Se separa la señal en I(t) y Q(t), desfasadas 90° entre sí.

    4. **Transmisión:**
    Se modula una portadora con ambas señales:
    $$s(t)=I(t)\cos(2\pi f_c t) - Q(t)\sin(2\pi f_c t)$$

    5. **Recepción y demodulación:**
    Se invierten los pasos anteriores para recuperar los bits.

    Este proceso ocurre constantemente, permitiendo enviar videos, mensajes o llamadas en tiempo real.
    """)

Writing 1_Conceptos_clave.py


In [ ]:
!mv 1_Conceptos_clave.py pages/

In [ ]:
%%writefile 2_Fases.py

import streamlit as st
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from scipy.signal import butter, lfilter, freqz, hilbert

st.set_page_config(layout="wide")
st.title("Simulaciones del Proyecto Final - Señales y Sistemas")

st.markdown("""
Esta sección presenta simulaciones claves para comprender la evolución de una señal en un sistema de comunicaciones moderno, desde su análisis espectral hasta su modulación y transmisión con ruido. Cada fase representa un paso fundamental en el procesamiento digital de señales.
""")

fase = st.sidebar.radio("Selecciona una fase:", [
    "Fase 1: Dominio de la Frecuencia",
    "Fase 2: Construcción de señales I/Q",
    "Fase 3: Modulación 16-QAM",
    "Fase 4: Sistema completo con canal ruidoso"
])

if fase == "Fase 1: Dominio de la Frecuencia":
    st.header("Fase 1: Dominio de la Frecuencia")
    st.markdown("""
    Se construye una señal sintética compuesta por varias frecuencias, se analiza su espectro con FFT, se filtra con un filtro paso bajo, y se visualiza la respuesta del filtro con un diagrama de Bode.
    """)

    f1 = st.slider("Frecuencia 1 (Hz)", 10, 100, 50)
    f2 = st.slider("Frecuencia 2 (Hz)", 100, 300, 150)
    f3 = st.slider("Frecuencia 3 (Hz)", 200, 500, 300)

    fs = 1000
    T = 1.0
    t = np.linspace(0, T, int(fs*T), endpoint=False)
    x = np.sin(2*np.pi*f1*t) + 0.5*np.sin(2*np.pi*f2*t) + 0.2*np.sin(2*np.pi*f3*t)

    col1, col2 = st.columns(2)

    with col1:
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.plot(t, x)
        ax.set_title("Señal Sintética en el Tiempo")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Amplitud")
        ax.grid()
        st.pyplot(fig)

    with col2:
        X = fft(x)
        f = fftfreq(len(t), 1/fs)
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.plot(f[:len(f)//2], 2/len(t)*np.abs(X[:len(X)//2]))
        ax.set_title("Espectro (Magnitud FFT)")
        ax.set_xlabel("Frecuencia [Hz]")
        ax.set_ylabel("Amplitud")
        ax.grid()
        st.pyplot(fig)

    col3, col4 = st.columns(2)

    with col3:
        b, a = butter(4, 0.2)
        w, h = freqz(b, a, worN=8000)
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.plot(fs * 0.5 * w / np.pi, 20 * np.log10(abs(h)))
        ax.set_title("Diagrama de Bode del Filtro")
        ax.set_xlabel("Frecuencia [Hz]")
        ax.set_ylabel("Ganancia [dB]")
        ax.grid()
        st.pyplot(fig)

    with col4:
        x_filt = lfilter(b, a, x)
        fig, ax = plt.subplots(figsize=(6, 3))
        ax.plot(t, x, label='Original')
        ax.plot(t, x_filt, label='Filtrada', alpha=0.8)
        ax.set_title("Señal Original vs Filtrada")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Amplitud")
        ax.legend()
        ax.grid()
        st.pyplot(fig)


elif fase == "Fase 2: Construcción de señales I/Q":
    st.header("Fase 2: Construcción de señales I/Q")
    st.markdown("""
    A partir de una señal senoidal, se genera su versión desfasada 90° mediante la transformada de Hilbert. Se visualizan ambas señales en el tiempo y como una constelación (diagrama de Lissajous), y se analiza su contenido espectral.
    """)

    freq = st.slider("Frecuencia de la señal base (Hz)", 1, 100, 2)
    fs = 1000
    T = 1.0
    t = np.linspace(0, T, int(fs*T), endpoint=False)
    I = np.cos(2 * np.pi * freq * t)
    Q = np.imag(hilbert(I))

    col1, col2 = st.columns(2)

    with col1:
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(t, I, label='I (coseno)', color='blue')
        ax.plot(t, Q, label='Q (Hilbert)', color='orange', linestyle='--')
        ax.set_title("Señales I y Q en el Tiempo")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Amplitud")
        ax.grid()
        ax.legend()
        st.pyplot(fig)

    with col2:
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(I, Q, color='purple')
        ax.set_title("Constelación I/Q (Lissajous)")
        ax.set_xlabel("I")
        ax.set_ylabel("Q")
        ax.grid()
        ax.axis("equal")
        st.pyplot(fig)

    # Espectro
    N = len(t)
    f = fftfreq(N, d=1/fs)
    mask = f > 0
    I_fft = fft(I)
    Q_fft = fft(Q)

    col_center = st.columns([1, 2, 1])[1]
    with col_center:
        fig, ax1 = plt.subplots(figsize=(6, 4))
        ax1.plot(f[mask], 20 * np.log10(np.abs(I_fft[mask])), label='I (coseno)', color='blue')
        ax1.plot(f[mask], 20 * np.log10(np.abs(Q_fft[mask])), label='Q (Hilbert)', color='orange')
        ax1.set_ylabel("Magnitud [dB]")
        ax1.set_xlabel("Frecuencia [Hz]")
        ax1.set_title("Espectro de Magnitud")
        ax1.grid(True)
        ax1.legend()
        st.pyplot(fig)


elif fase == "Fase 3: Modulación 16-QAM":
    st.header("Fase 3: Modulación 16-QAM")
    st.markdown("""
    En esta fase se implementa una modulación 16-QAM con mapeo Gray. Se genera la señal I/Q modulada en portadoras coseno/seno y se visualiza tanto la señal modulada como su espectro y constelación. El usuario puede modificar la frecuencia de muestreo `fs` para ver su efecto.
    """)

    fs = st.slider("Frecuencia de muestreo fs (Hz)", 5000, 20000, 10000, step=1000)

    N = 200  # Número de símbolos
    T_symbol = 0.01
    f_carrier = 1000
    samples_per_symbol = int(fs * T_symbol)

    def qam16_gray_mapper(symbols):
        mapping = {
            0: (-3, -3),  1: (-3, -1),  2: (-3, +3),  3: (-3, +1),
            4: (-1, -3),  5: (-1, -1),  6: (-1, +3),  7: (-1, +1),
            8: (+3, -3),  9: (+3, -1), 10: (+3, +3), 11: (+3, +1),
            12: (+1, -3),13: (+1, -1),14: (+1, +3),15: (+1, +1)
        }
        return np.array([mapping[s] for s in symbols])

    symbols = np.random.randint(0, 16, N)
    mapped = qam16_gray_mapper(symbols)
    I_seq, Q_seq = mapped[:, 0], mapped[:, 1]
    I_t = np.repeat(I_seq, samples_per_symbol)
    Q_t = np.repeat(Q_seq, samples_per_symbol)
    t = np.linspace(0, T_symbol * N, samples_per_symbol * N, endpoint=False)
    carrier_I = np.cos(2 * np.pi * f_carrier * t)
    carrier_Q = np.sin(2 * np.pi * f_carrier * t)
    qam_signal = I_t * carrier_I + Q_t * carrier_Q

    col1, col2 = st.columns(2)

    with col1:
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(t[:5*samples_per_symbol], qam_signal[:5*samples_per_symbol])
        ax.set_title("Señal QAM Modulada (5 símbolos)")
        ax.set_xlabel("Tiempo [s]")
        ax.set_ylabel("Amplitud")
        ax.grid(True)
        st.pyplot(fig)

    with col2:
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.scatter(I_seq, Q_seq, alpha=0.7)
        ax.set_title("Constelación 16-QAM (Gray)")
        ax.set_xlabel("Componente I")
        ax.set_ylabel("Componente Q")
        ax.grid(True)
        ax.axis("equal")
        st.pyplot(fig)

    n = len(qam_signal)
    f = fftfreq(n, 1/fs)
    spectrum = np.abs(fft(qam_signal)) / n

    col_center = st.columns([1, 2, 1])[1]
    with col_center:
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.plot(f[:n//2], spectrum[:n//2])
        ax.set_title("Espectro de Magnitud de la Señal QAM")
        ax.set_xlabel("Frecuencia [Hz]")
        ax.set_ylabel("Magnitud")
        ax.grid(True)
        st.pyplot(fig)


elif fase == "Fase 4: Sistema completo con canal ruidoso":
    st.header("Fase 4: Sistema completo con canal ruidoso")
    st.markdown("""
    Se construye un sistema de transmisión completo con modulación QAM de orden variable, adición de ruido AWGN y demodulación para recuperar la constelación estimada.
    """)

    EbN0_dB = st.slider("SNR (Eb/N0) [dB]", 0, 50, 25)
    orden_qam = st.selectbox("Orden QAM", [16, 64, 256], index=0)

    fs = 10000
    T_symbol = 0.01
    f_carrier = 1000
    samples_per_symbol = int(fs * T_symbol)
    N = 200
    bits_per_symbol = int(np.log2(orden_qam))

    def qam_gray_mapper(symbols, M):
        m = int(np.sqrt(M))
        levels = np.arange(-m + 1, m, 2)
        mapping = []
        for s in symbols:
            row = s // m
            col = s % m
            I = levels[col]
            Q = levels[row]
            mapping.append((I, Q))
        return np.array(mapping)

    def awgn(x, EbN0_dB, bits_per_symbol=4, fs=1):
        Es = np.mean(np.abs(x)**2)
        Eb = Es / bits_per_symbol
        N0 = Eb / (10**(EbN0_dB / 10))
        sigma = np.sqrt(N0 * fs / 2)
        ruido = np.random.normal(0, sigma, len(x))
        return x + ruido

    def demodulate(signal, fs, f_carrier, T_symbol, N):
        sps = int(T_symbol * fs)
        t = np.linspace(0, T_symbol, sps, endpoint=False)
        I_carrier = np.cos(2 * np.pi * f_carrier * t)
        Q_carrier = np.sin(2 * np.pi * f_carrier * t)
        I_demod, Q_demod = [], []
        for i in range(N):
            segmento = signal[i*sps:(i+1)*sps]
            I = 2 * np.sum(segmento * I_carrier) / sps
            Q = 2 * np.sum(segmento * Q_carrier) / sps
            I_demod.append(I)
            Q_demod.append(Q)
        return np.array(I_demod), np.array(Q_demod)

    symbols = np.random.randint(0, orden_qam, N)
    mapped = qam_gray_mapper(symbols, orden_qam)
    I_seq, Q_seq = mapped[:, 0], mapped[:, 1]

    t_symbol = np.linspace(0, T_symbol, samples_per_symbol, endpoint=False)
    modulated = np.concatenate([
        i * np.cos(2 * np.pi * f_carrier * t_symbol) +
        q * np.sin(2 * np.pi * f_carrier * t_symbol)
        for i, q in zip(I_seq, Q_seq)
    ])

    modulada_con_ruido = awgn(modulated, EbN0_dB, bits_per_symbol=bits_per_symbol, fs=fs)
    I_hat, Q_hat = demodulate(modulada_con_ruido, fs, f_carrier, T_symbol, N)

    # Columnas para constelaciones
    col1, col2 = st.columns(2)
    with col1:
        fig1, ax1 = plt.subplots(figsize=(6, 4))
        ax1.scatter(I_seq, Q_seq, alpha=0.8, edgecolors='k')
        ax1.set_title(f"Constelación Original ({orden_qam}-QAM)")
        ax1.set_xlabel("Componente I")
        ax1.set_ylabel("Componente Q")
        ax1.grid(True)
        ax1.axis("equal")
        st.pyplot(fig1)

    with col2:
        fig2, ax2 = plt.subplots(figsize=(6, 4))
        ax2.scatter(I_hat, Q_hat, alpha=0.8, color='darkorange', edgecolors='k')
        ax2.set_title("Constelación con Ruido AWGN")
        ax2.set_xlabel("I estimado")
        ax2.set_ylabel("Q estimado")
        ax2.grid(True)
        ax2.axis("equal")
        st.pyplot(fig2)

    # Señal en el tiempo - centrada
    samples_plot = 5 * samples_per_symbol
    t_plot = np.linspace(0, 5 * T_symbol, samples_plot, endpoint=False)

    col3 = st.columns([1, 2, 1])[1]
    with col3:
        fig3, ax3 = plt.subplots(figsize=(6, 4))
        ax3.plot(t_plot, modulated[:samples_plot], label='Señal Modulada')
        ax3.plot(t_plot, modulada_con_ruido[:samples_plot], label='Señal con Ruido', alpha=0.7)
        ax3.set_title("Señal en el Tiempo (Primeros 5 Símbolos)")
        ax3.set_xlabel("Tiempo [s]")
        ax3.set_ylabel("Amplitud")
        ax3.grid(True)
        ax3.legend()
        st.pyplot(fig3)





Writing 2_Fases.py


In [ ]:
!mv 2_Fases.py pages/

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

#Ejecutar Streamlit
!streamlit run 0_Introducción.py &>/content/logs.txt &

#Exponer el puerto 8501 con Cloudflare Tunnel
!cloudflared tunnel --url http://localhost:8501 > /content/cloudflared.log 2>&1 &

#Leer la URL pública generada por Cloudflare
import time
time.sleep(5)  # Esperar que se genere la URL

import re
found_context = False  # Indicador para saber si estamos en la sección correcta

with open('/content/cloudflared.log') as f:
    for line in f:
        #Detecta el inicio del contexto que nos interesa
        if "Your quick Tunnel has been created" in line:
            found_context = True

        #Busca una URL si ya se encontró el contexto relevante
        if found_context:
            match = re.search(r'https?://\S+', line)
            if match:
                url = match.group(0)  #Extrae la URL encontrada
                print(f'Tu aplicación está disponible en: {url}')
                break  #Termina el bucle después de encontrar la URL

--2025-07-22 14:06:05--  https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64 [following]
--2025-07-22 14:06:06--  https://github.com/cloudflare/cloudflared/releases/download/2025.7.0/cloudflared-linux-amd64
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/106867604/37d2bad8-a2ed-4b93-8139-cbb15162d81d?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-07-22T15%3A00%3A37Z&rscd=attachment%3B+filename%3Dcloudflared-linux-amd64&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-07-22T1

In [ ]:
import os

res = input("Digite (1) para finalizar la ejecución del Dashboard: ")

if res.upper() == "1":
    os.system("pkill streamlit")  # Termina el proceso de Streamlit
    print("El proceso de Streamlit ha sido finalizado.")

Digite (1) para finalizar la ejecución del Dashboard: 1
El proceso de Streamlit ha sido finalizado.
